In [253]:
import pandas as pd

In [254]:
 df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
 ....: 'data1': range(7)})

In [255]:
 df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
 ....: 'data2': range(3)})

In [256]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [257]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [258]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [259]:
 df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
 ....: 'data1': range(7)})

In [260]:
 df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
 ....: 'data2': range(3)})

In [261]:
df3

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [262]:
df4

,data2,rkey
0,0,a
1,1,b
2,2,d


In [263]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [264]:
 df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
 ....: 'data1': range(6)})

In [265]:
 df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
 ....: 'data2': range(5)})

In [266]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [267]:
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [268]:
pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [269]:
 left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
 ....: 'key2': ['one', 'two', 'one'],
 ....: 'lval': [1, 2, 3]})

In [270]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [271]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
 ....: 'key2': ['one', 'one', 'one', 'two'],
 ....: 'rval': [4, 5, 6, 7]})

In [272]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [273]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [274]:
 left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
 ....: 'value': range(6)})

In [275]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [276]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [277]:
right1

,group_val
a,3.5
b,7.0


In [278]:
 pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [279]:
 pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [280]:
import numpy as np
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
'key2': [2000, 2001, 2002, 2001, 2002],
'data': np.arange(5.)})

In [281]:
 righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
  index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
  [2001, 2000, 2000, 2000, 2001, 2002]],
  columns=['event1', 'event2'])

In [282]:
lefth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [283]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [284]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [285]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
 ....: columns=['Ohio', 'Nevada'])

In [286]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
 ....: index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])

In [287]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [288]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [289]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [290]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])

In [291]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [292]:
s1 = pd.Series([0, 1], index=['a', 'b']);
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e']);
s3 = pd.Series([5, 6], index=['f', 'g']);

In [293]:
 s4 = pd.concat([s1 * 5, s3])

In [294]:
s4

a    0
b    5
f    5
g    6
dtype: int64

In [295]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [296]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


In [297]:
 a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
 ....: index=['f', 'e', 'd', 'c', 'b', 'a'])

In [298]:
 b = pd.Series(np.arange(len(a), dtype=np.float64),
 ....: index=['f', 'e', 'd', 'c', 'b', 'a'])

In [299]:
 b[-1] = np.nan

In [300]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [301]:
b[:-2]

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64

In [302]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [303]:
 b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [304]:
 data = pd.DataFrame(np.arange(6).reshape((2, 3)),
 ....: index=pd.Index(['Ohio', 'Colorado'], name='state'),
 ....: columns=pd.Index(['one', 'two', 'three'], name='number'))

In [305]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [306]:
result = data.stack()

In [307]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [308]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [309]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

In [310]:
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [311]:
s1

a    0
b    1
c    2
d    3
dtype: int64

In [312]:
s2

c    4
d    5
e    6
dtype: int64

In [313]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [314]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [315]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [316]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [317]:
 df = pd.DataFrame({'left': result, 'right': result + 5},
 .....: columns=pd.Index(['left', 'right'], name='side'))

In [318]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [319]:
df.unstack('state').stack('side')

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

In [320]:
 df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
 ....: 'b': [np.nan, 2., np.nan, 6.],
 ....: 'c': range(2, 18, 4)})

In [321]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [322]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
 ....: 'b': [np.nan, 3., 4., 6., 8.]})

In [323]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [324]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [325]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
 .....: 'k2': [1, 1, 2, 3, 3, 4, 4]})

In [326]:
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [327]:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [328]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [329]:
data['v1']=range(7)

In [330]:
data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6


In [331]:
data.drop_duplicates(['k1', 'k2'])

,k1,k2,v1
0,one,1,0
2,one,2,2
3,two,3,3
5,two,4,5


In [332]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


In [333]:
 data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
 .....: 'corned beef', 'Bacon', 'pastrami', 'honey ham',
 .....: 'nova lox'],
 .....: 'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [334]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [335]:
 data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [336]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [337]:
data = np.random.rand(20)

In [338]:
data

array([0.04603641, 0.53457027, 0.81868953, 0.39509542, 0.34641123,
       0.05340884, 0.93324835, 0.10275095, 0.53614715, 0.00426885,
       0.97467944, 0.44289236, 0.65158907, 0.61562033, 0.53089883,
       0.13941997, 0.48344916, 0.11734259, 0.07557764, 0.21863608])

In [339]:
pd.cut(data, 4, precision=3)

[(0.0033, 0.247], (0.489, 0.732], (0.732, 0.975], (0.247, 0.489], (0.247, 0.489], ..., (0.0033, 0.247], (0.247, 0.489], (0.0033, 0.247], (0.0033, 0.247], (0.0033, 0.247]]
Length: 20
Categories (4, object): [(0.0033, 0.247] < (0.247, 0.489] < (0.489, 0.732] < (0.732, 0.975]]

In [340]:
pd.cut(data, 4, precision=2)

[(0.0033, 0.25], (0.49, 0.73], (0.73, 0.97], (0.25, 0.49], (0.25, 0.49], ..., (0.0033, 0.25], (0.25, 0.49], (0.0033, 0.25], (0.0033, 0.25], (0.0033, 0.25]]
Length: 20
Categories (4, object): [(0.0033, 0.25] < (0.25, 0.49] < (0.49, 0.73] < (0.73, 0.97]]

In [341]:
data = np.random.randn(1000)

In [342]:
cats = pd.qcut(data, 4)
cats

[(-0.699, -0.0238], (0.658, 3.526], [-3.646, -0.699], (-0.0238, 0.658], [-3.646, -0.699], ..., (-0.0238, 0.658], (-0.699, -0.0238], (0.658, 3.526], [-3.646, -0.699], [-3.646, -0.699]]
Length: 1000
Categories (4, object): [[-3.646, -0.699] < (-0.699, -0.0238] < (-0.0238, 0.658] < (0.658, 3.526]]

In [343]:
 np.random.seed(12345)

In [344]:
data = pd.DataFrame(np.random.randn(1000,4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [345]:
 sampler = np.random.permutation(5)

In [346]:
sampler

array([1, 0, 2, 3, 4])

In [347]:
df = pd.DataFrame(np.arange(5 * 4).reshape(5, 4))

In [348]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [349]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [350]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
 .....: 'data1': range(6)})

In [351]:
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [352]:
df[['data1']]

,data1
0,0
1,1
2,2
3,3
4,4
5,5


In [353]:
df['data1']

0    0
1    1
2    2
3    3
4    4
5    5
Name: data1, dtype: int32

In [354]:
mnames = ['movie_id', 'title', 'genres']

In [355]:
movies = pd.read_table('ch02\movielens\movies.dat', sep='::', header=None, names=mnames)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [356]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [357]:
dummies = pd.DataFrame(np.zeros((len(movies), len(movies.genres))), columns=movies.genres)

In [358]:
for i, gen in enumerate(movies.genres):
    dummies.ix[i, gen.split('|')] = 1

In [359]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [360]:
movies_windic[:10]

,movie_id,title,genres,Genre_Animation|Children's|Comedy,Genre_Adventure|Children's|Fantasy,Genre_Comedy|Romance,Genre_Comedy|Drama,Genre_Comedy,Genre_Action|Crime|Thriller,Genre_Comedy|Romance,...,Genre_Comedy,Genre_Comedy|Drama,Genre_Adventure|Animation|Children's,Genre_Action|Drama|Thriller,Genre_Thriller,Genre_Comedy,Genre_Drama,Genre_Drama,Genre_Drama,Genre_Drama|Thriller
0,1,Toy Story (1995),Animation|Children's|Comedy,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,6,Heat (1995),Action|Crime|Thriller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,7,Sabrina (1995),Comedy|Romance,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,8,Tom and Huck (1995),Adventure|Children's,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,Sudden Death (1995),Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,GoldenEye (1995),Action|Adventure|Thriller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [361]:
val = 'a,b, guido'

In [362]:
val.index(',')

1

In [363]:
val.find(',')

1

In [364]:
val.find(':')

-1

In [365]:
val.replace(',', '::')

'a::b:: guido'

In [366]:
import re

In [367]:
text = "foo bar\t baz \tqux"

In [368]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [369]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [370]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [371]:
m = regex.search(text)
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [372]:
m.start()

5

In [373]:
m.end()

20

In [377]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [379]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [380]:
regex = re.compile(pattern, flags=re.IGNORECASE)

In [383]:
m = regex.match('wesm@bright.net')

In [384]:
m

<_sre.SRE_Match object; span=(0, 15), match='wesm@bright.net'>

In [385]:
m.groups()

('wesm', 'bright', 'net')

In [387]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



In [391]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
 .....: 'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)

In [393]:
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

In [395]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [397]:
matches = data.str.match(pattern, flags=re.IGNORECASE)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: In future versions of pandas, match will change to always return a bool indexer.
  if __name__ == '__main__':


In [398]:
matches

Dave     (dave, google, com)
Rob        (rob, gmail, com)
Steve    (steve, gmail, com)
Wes                      NaN
dtype: object

In [399]:
matches.str.get(1)

Dave     google
Rob       gmail
Steve     gmail
Wes         NaN
dtype: object

In [ ]:
mat